In [110]:
import torch
import pyro
import pyro.distributions as dist
from torch.distributions import constraints
import numpy as np

In [2]:
# model
def scale(guess):
    weight = pyro.sample("weight", dist.Normal(guess, 1.0))
    return pyro.sample("measurement", dist.Normal(weight, 0.75))


In [3]:
scale(torch.tensor(3.3))

tensor(3.8493)

##### for condition
This function is only looks at weight, that satisfy `measurement==9.5`:

In [4]:
# condition0
def conditioned_scale_obs(guess):  # equivalent to conditioned_scale above
    # print("conditioned guess:", guess)
    weight = pyro.sample("weight", dist.Normal(guess, 1.))
     # here we condition on measurement == 9.5
    return pyro.sample("measurement", dist.Normal(weight, 0.75), obs=9.5)

##### for guide
i.e. approximation for weight's posterior distribution `quides = q(z=weight) ~ p(z=weight|x=mesurment)` (will be with use of ELBO):

In [5]:
# guide0
from torch.distributions import constraints

def scale_parametrized_guide(guess):
    
    a = pyro.param("a", torch.tensor(guess))
    b = pyro.param("b", torch.tensor(1.), constraint=constraints.positive)
    # print("guide a:", a)
    return pyro.sample("weight", dist.Normal(a, torch.abs(b)))

In [6]:
scale_parametrized_guide(torch.tensor(2.2))

/home/valdecar/anaconda3/envs/math/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


tensor(2.5861, grad_fn=<AddBackward0>)

##### for params

In [7]:
print(pyro.get_param_store().keys())

dict_keys(['a', 'b'])


In [8]:
pyro.clear_param_store()

In [9]:
print(pyro.get_param_store().keys())

dict_keys([])


##### for inference
Take such params (`a` and `b`) so they can produce `weight` (using `scale_parametrized_guide`) which satisfy condition (i.e. `conditioned_scale_ops`) (condition used as model). So we knew about `p(x|z)` and will try to find `p(z|x)`:

In [11]:
guess = 8.5

pyro.clear_param_store()
svi = pyro.infer.SVI(model=conditioned_scale_obs,
                     guide=scale_parametrized_guide,
                     optim=pyro.optim.SGD({"lr": 0.001, "momentum":0.1}),
                     loss=pyro.infer.Trace_ELBO())


losses, a,b  = [], [], []
num_steps = 250  # 2500
for t in range(num_steps):
    losses.append(svi.step(guess))
    a.append(pyro.param("a").item())
    b.append(pyro.param("b").item())

In [12]:
print(len(a))
print(a[-1])

250
8.809281349182129


##### results:

In [13]:
import matplotlib.pyplot as plt


In [14]:
%matplotlib
plt.plot(losses)
plt.title("ELBO")
plt.xlabel("step")
plt.ylabel("loss");
print('a = ',pyro.param("a").item())
print('b = ', pyro.param("b").item())

Using matplotlib backend: Qt5Agg
a =  8.809281349182129
b =  0.7539201378822327


In [16]:
%matplotlib
plt.subplot(1,2,1)
plt.plot([0,num_steps],[9.14,9.14], 'k:')
plt.plot(a)
plt.ylabel('a')

plt.subplot(1,2,2)
plt.ylabel('b')
plt.plot([0,num_steps],[0.6,0.6], 'k:')
plt.plot(b)
plt.tight_layout()

Using matplotlib backend: Qt5Agg


In [9]:
data = torch.zeros(10)
data[0:6] = torch.ones(6)


# for vectorized, sampled, independed:
def model13():
    alpha0 = torch.tensor(10.0)
    beta0 = torch.tensor(10.0)

    f = pyro.sample("latent_fairness", dist.Beta(alpha0, beta0)) 

    # vectorized, sampled, independent:
    with pyro.plate("data_loop", size=10, subsample_size=5) as ind:
        dd = dist.Bernoulli(f).expand([5])
        print("dd.batch_shape:")
        print(dd.batch_shape)
        print("dd.event_shape:")
        print(dd.event_shape)
        print("ind:")
        print(ind)
        a = pyro.sample("obs", dd, obs=data.index_select(0, ind))
        print("a:")
        print(a)


In [10]:
model13()

dd.batch_shape:
torch.Size([5])
dd.event_shape:
torch.Size([])
ind:
tensor([8, 6, 7, 5, 0])
a:
tensor([0., 0., 0., 1., 1.])


In [117]:

# for vectorized, sampled, depended:
def model23():
    # alpha0 = torch.tensor(10.0)
    # beta0 = torch.tensor(10.0)
    mu = torch.tensor(0.5)
    sigma = torch.tensor(0.1)

    p = pyro.sample("latent_fairness", dist.Normal(mu, sigma))
    print("p = ", p)
    
    # vectorized, sampled, dependent:
    with pyro.plate("data_loop", size=10, subsample_size=5) as ind:
        dd = dist.Bernoulli(p).expand([5, 2]).to_event(1)
        print("dd.batch_shape:")
        print(dd.batch_shape)
        print("dd.event_shape:")
        print(dd.event_shape)
        x = dd.sample()
        print("x = ", x)
        print("prob(x) = ", torch.exp(dd.log_prob(x)))
        print("accurate: ",
              torch.tensor([(p if x0 else 1-p)*(p if x1 else 1-p)
                            for x0, x1 in x]))
        
        print("ind:")
        print(ind)
        a = pyro.sample("obs", dd, obs=data.index_select(0, ind))
        print("a:")
        print(a)


In [118]:
model23()

p =  tensor(0.5121)
dd.batch_shape:
torch.Size([5])
dd.event_shape:
torch.Size([2])
x =  tensor([[0., 0.],
        [1., 0.],
        [0., 1.],
        [1., 1.],
        [1., 0.]])
prob(x) =  tensor([0.2380, 0.2499, 0.2499, 0.2623, 0.2499])
accurate:  tensor([0.2380, 0.2499, 0.2499, 0.2623, 0.2499])
ind:
tensor([6, 2, 9, 8, 0])
a:
tensor([0., 1., 0., 0., 1.])


##### for 
conditionaly depended:

In [93]:
# Bernulli:
p = 0.7
dd = dist.Bernoulli(p*torch.ones((3,2))).to_event(1)
x = dd.sample()
print("x = ", x)
print("prob(x) = ", torch.exp(dd.log_prob(x)))
print("accurate: ", torch.tensor([(p if x0 else 1-p)*(p if x1 else 1-p)
                                  for x0, x1 in x]))

x =  tensor([[1., 0.],
        [1., 1.],
        [1., 1.]])
prob(x) =  tensor([0.2100, 0.4900, 0.4900])
accurate:  tensor([0.2100, 0.4900, 0.4900])


In [95]:
# Normal:
dd = dist.Normal(0.0*torch.ones((3,2)),
                 1.0*torch.ones((3,2))).to_event(1)
x = dd.sample()
print("x = ", x)
print("prob(x) = ", torch.exp(dd.log_prob(x)))

f = lambda x: 1/np.sqrt(2*np.pi) * np.exp(-0.5*np.power(x,2))
print("accurate: ", torch.tensor([f(x0)*f(x1) for x0, x1 in x]))

x =  tensor([[-1.0040, -0.5045],
        [-1.2116, -0.4331],
        [-0.5154,  1.5380]])
prob(x) =  tensor([0.0847, 0.0696, 0.0427])
accurate:  tensor([0.0847, 0.0696, 0.0427], dtype=torch.float64)


In [78]:
# dd.to_event??
# isinstance(dd, torch.distributions.Independent)
# torch.distributions.Independent?

##### for 
`rand`

In [37]:
torch.rand((3,2))

tensor([[0.7280, 0.2271],
        [0.5586, 0.0519],
        [0.9976, 0.1992]])

##### for 
`dist.log_prob`

In [61]:
d = torch.distributions.Normal(torch.tensor([0.0]),
                               torch.tensor([1.0]))

In [64]:
f = lambda x: 1/np.sqrt(2*np.pi) * np.exp(-0.5*np.power(x,2))

In [66]:
print("accurate: ", f(0))
print("pyros: ", torch.exp(d.log_prob(0.0)))

accurate:  0.3989422804014327
pyros:  tensor([0.3989])


In [75]:
d = torch.distributions.Bernoulli(0.7)
print(torch.exp(d.log_prob(0)))

tensor(0.3000)


##### for 
`plate`

In [ ]:
x_axis = pyro.plate("x", 3, dim=-1)
y_axis = pyro.plate("y", 2, dim=-2)
with x_axis:
    x = pyro.sample("x", dist.Normal(0, 1))
with y_axis:
    y = pyro.sample("y", dist.Normal(0, 1))
print("x: ", x)
print("y: ", y)